# Deploying a Decision Optimization model with Watson Machine Learning

This notebook shows you how to deploy a Decision Optimization model, create and monitor jobs, and get solutions using the Watson Machine Learning Python Client.


# Set up the Watson Machine Learning client

Before you use the sample code in this notebook, you need to:

    create a Watson Machine Learning (WML) Service instance. A free plan is offered and information about how to create the instance can be found here.

Install and then import the Watson Machine Learning client library. This notebook uses the preview Python client based on v4 of Watson Machine Learning APIs.

Important Do not load both Python client libraries into a notebook.


In [50]:
# Uninstall the Watson Machine Learning client Python client based on v3 APIs

!pip uninstall watson-machine-learning-client -y

In [51]:
# Install the WML client API

!pip install watson-machine-learning-client-V4

In [52]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

Create a client instance

Use your Watson Machine Learning credentials.

In [53]:
# The code was removed by Watson Studio for sharing.

In [54]:
%mkdir model

mkdir: cannot create directory ‘model’: File exists


In [55]:
%%writefile model/main.py

from docplex.util.environment import get_environment
from os.path import splitext
import pandas
from six import iteritems

def get_all_inputs():
    '''Utility method to read a list of files and return a tuple with all
    read data frames.
    Returns:
        a map { datasetname: data frame }
    '''
    result = {}
    env = get_environment()
    for iname in [f for f in os.listdir('.') if splitext(f)[1] == '.csv']:
        with env.get_input_stream(iname) as in_stream:
            df = pandas.read_csv(in_stream)
            datasetname, _ = splitext(iname)
            result[datasetname] = df
    return result

def write_all_outputs(outputs):
    '''Write all dataframes in ``outputs`` as .csv.

    Args:
        outputs: The map of outputs 'outputname' -> 'output df'
    '''
    for (name, df) in iteritems(outputs):
        csv_file = '%s.csv' % name
        print(csv_file)
        with get_environment().get_output_stream(csv_file) as fp:
            if sys.version_info[0] < 3:
                fp.write(df.to_csv(index=False, encoding='utf8'))
            else:
                fp.write(df.to_csv(index=False).encode(encoding='utf8'))
    if len(outputs) == 0:
        print("Warning: no outputs written")
        

Overwriting model/main.py


In [56]:
%%writefile -a model/main.py

#dd-cell
import pandas as pd
import numpy as np

# Load CSV files into inputs dictionnary
inputs = get_all_inputs()
df_day = inputs['day']

df_machine = inputs['machine']

df_predicted_failure = inputs['predicted_failure']
df_predicted_failure = df_predicted_failure.set_index(['machine', 'day'])

df_planned_production = inputs['planned_production']
df_planned_production = df_planned_production.set_index(['machine', 'day'])

df_fixed_maintenance = inputs['fixed_maintenance']
df_fixed_maintenance = df_fixed_maintenance.set_index(['machine', 'day'])

df_parameters = inputs['parameters']

# first global collections to iterate upon
all_machines = df_machine['id'].values

all_days = df_day['id'].values

data_cumul_failure = []
for machine in all_machines:
    for i, d in np.ndenumerate(all_days):
        cumul = 0
        for i2, d2 in np.ndenumerate(all_days):
            if i2==i:
                break
            cumul += int(df_predicted_failure.failure[machine, d2])
        data_cumul_failure.append((machine, d, cumul))

df_cumul_failure = pd.DataFrame(data_cumul_failure, columns=['machine', 'day', 'cumul_failure'])
df_cumul_failure=df_cumul_failure.set_index(['machine', 'day'])
#dd-cell
from docplex.mp.environment import Environment
env = Environment()
env.print_information()    
#dd-cell
from docplex.mp.model import Model
mdl = Model(name="PredictiveMaintenance")
#dd-cell

# the variables.
production = mdl.continuous_var_matrix(keys1=all_machines, keys2=all_days, name=lambda ns: "Production_%s_%s" % (ns[0],ns[1]))
df_production = pd.DataFrame({'production': production})
df_production.index.names=['all_machines', 'all_days']

maintenance = mdl.binary_var_matrix(keys1=all_machines, keys2=all_days, name=lambda ns: "Maintenance_%s_%s" % (ns[0],ns[1]))
df_maintenance = pd.DataFrame({'maintenance': maintenance})
df_maintenance.index.names=['all_machines', 'all_days']
#dd-cell
# take into account fixed maintenance events
fixm = int(df_parameters[df_parameters.id=='fix_maintenance']['value'])
if fixm == 1:
    for machine in all_machines:
        for day in all_days:
            if (df_fixed_maintenance.maintenance[machine, day] == 1):
                mdl.add_constraint(maintenance[machine, day] == 1)
#dd-cell
for machine in all_machines:       
    maintenance_loss = int(df_machine[df_machine.id==machine]['maintenance loss'])/100.
    capacity = int(df_machine[df_machine.id==machine]['capacity'])
    for day in all_days:   
        prod = df_planned_production.production[machine, day]
        #mdl.add_if_then( maintenance[machine, day] == 1, production[machine, day]== 0 )
        #mdl.add_if_then( maintenance[machine, day] == 0, production[machine, day]== df_production[df_production.machine==machine][df_production.day==day] )
        if (prod <= capacity*(1-maintenance_loss)):
            mdl.add_constraint( production[machine, day] == prod )
        else:
            mdl.add_constraint( production[machine, day] == prod - (prod-capacity*(1-maintenance_loss))*maintenance[machine, day])
        
#dd-cell
# One maintenance per machine
for machine in all_machines:       
    mdl.add_constraint( mdl.sum(maintenance[machine, day] for day in all_days) == 1)
    
maintenance_crew_size = int(df_parameters[df_parameters.id=='maintenance crew size']['value'])

# One maintenance at a time
for day in all_days:       
    mdl.add_constraint( mdl.sum(maintenance[machine, day] for machine in all_machines) <= maintenance_crew_size)

data_cost_maintenance = []
data_cost_maintenance_detail = []
cost_kpis = []
# Cost of repair

for machine in all_machines:           
    #print machine
    life = int(df_machine[df_machine.id==machine]['remaining life'])
    capacity = int(df_machine[df_machine.id==machine]['capacity'])
    cost_of_maintenance = int(df_machine[df_machine.id==machine]['cost of maintenance'])
    maintenance_loss = int(df_machine[df_machine.id==machine]['maintenance loss'])/100.
    cost_of_repair = int(df_machine[df_machine.id==machine]['cost of repair'])
    repair_loss = int(df_machine[df_machine.id==machine]['repair loss'])/100.
    loss_per_life_day = int(df_machine[df_machine.id==machine]['loss per life day'])
    production_value_unit = int(df_machine[df_machine.id==machine]['production value unit'])
    
    previous_day = None
    for i, day in np.ndenumerate(all_days):
        cost = 0;
        prob_break_before = 0
        fail_prod_cost = 0
        repair_cost = 0
        maint_cost = 0
        prod_cost = 0
        early_maint_cost = 0
        
        if (previous_day != None):
            prob_break_before = int(df_cumul_failure.cumul_failure[machine, previous_day])/100.
        previous_day = day
        
        #print prob_break_before
        
        # Cost of lost production if failure before maintenance
        for i2, day2 in np.ndenumerate(all_days):
            if (i2==i):
                break
            prob_break_day2 = int(df_predicted_failure.failure[machine, day2])/100.
            production_day2 = int(df_planned_production.production[machine, day2])
            if (production_day2 > capacity*(1-repair_loss)):
                temp = production_value_unit*prob_break_day2*(production_day2 - capacity*(1-repair_loss))
                fail_prod_cost += temp
                cost += temp
                
            
        # Cost of repair if breaking before maintenance
        repair_cost = cost_of_repair*prob_break_before
        cost += repair_cost
        
        
        # Cost of maintenance
        maint_cost = cost_of_maintenance*(1-prob_break_before)
        cost += maint_cost
        
        # Cost of lost production for maintenance
        production_day = int(df_planned_production.production[machine, day])
        if (production_day > capacity*(1-maintenance_loss)):
            prod_cost = production_value_unit*(production_day - capacity*(1-maintenance_loss))
            cost += prod_cost
            
        
        # Cost of maintenance too early
        early_maint_cost = loss_per_life_day*max(life-i[0], 0)
        cost += early_maint_cost
        
        #print cost
        data_cost_maintenance.append((machine, day, cost))
        data_cost_maintenance_detail.append((machine, day, fail_prod_cost, repair_cost, maint_cost, prod_cost, early_maint_cost))
        
        cost_kpis.append(cost*maintenance[machine, day])
        
cost_kpi = mdl.sum(cost_kpis)
mdl.add_kpi(cost_kpi, "Cost")

df_cost_maintenance = pd.DataFrame(data_cost_maintenance, columns=['machine', 'day', 'cost_maintenance'])
df_cost_maintenance_detail = pd.DataFrame(data_cost_maintenance_detail, columns=['machine', 'day', 'fail_prod_cost', 'repair_cost', 'maint_cost', 'prod_cost', 'early_maint_cost'])
#print df_cost_maintenance
#dd-cell
total_planned_production = mdl.sum(df_planned_production.production)
mdl.add_kpi(total_planned_production, "Total Planned Production")
total_production = mdl.sum(df_production.production)
mdl.add_kpi(total_production, "Total Production")
#dd-cell
strategy = int(df_parameters[df_parameters.id=='strategy']['value'])

if (strategy == 1):
    mdl.minimize(cost_kpi)
else:
    early = 10
    late = 1000
    temp = []     
    for machine in all_machines:           
        
        last_day = None
        for i, day in np.ndenumerate(all_days):
            last_day = day;
            cumul_failure = int(df_cumul_failure.cumul_failure[machine, day])
            if (cumul_failure > 0):                            
                temp.append(late * maintenance[machine, day] )
            else:
                temp.append(early * i[0] * maintenance[machine, day] )
        
    late_kpi = mdl.sum(temp)
    mdl.minimize(late_kpi)
#dd-cell
s = mdl.solve(log_output=True)
assert s, "solve failed"
mdl.report()

all_kpis = [(kp.name, kp.compute()) for kp in mdl.iter_kpis()]
df_kpis = pd.DataFrame(all_kpis, columns=['kpi', 'value'])
    
#dd-cell
df_production = df_production.production.apply(lambda v: v.solution_value)
#df_production
#dd-cell
df_maintenance = df_maintenance.maintenance.apply(lambda v: v.solution_value)
#df_maintenance
#dd-cell
df_production = df_production.to_frame()
df_production['machine'] = df_production.index.get_level_values('all_machines') 
df_production['day'] = df_production.index.get_level_values('all_days') 
df_production.columns = ['production', 'machine', 'day'] 

# track production shortage = gap between planned production and actual
prod_shortage = []
for m in all_machines:
   for d in all_days:
      prod_shortage.append((m,d,df_planned_production.production[m,d]-df_production.production[m,d]))

df_production_shortage = pd.DataFrame(prod_shortage, columns=['machine', 'day', 'production_shortage'])
# end track production shortage

df_production = df_production.reset_index(drop=True)

df_maintenance = df_maintenance.to_frame()
df_maintenance['machine'] = df_maintenance.index.get_level_values('all_machines') 
df_maintenance['day'] = df_maintenance.index.get_level_values('all_days') 
df_maintenance.columns = ['maintenance', 'machine', 'day'] 

#track resource load (number of machines maintained each day)
res_load = []
for d in all_days:
    load = 0
    for m in all_machines:
        load += int(df_maintenance.maintenance[m,d])
    res_load.append((d,load))
df_resource_load = pd.DataFrame(res_load, columns=['day', 'load'])
# end track resource load 

df_cumul_failure['machine'] = df_cumul_failure.index.get_level_values('machine')
df_cumul_failure['day'] = df_cumul_failure.index.get_level_values('day')
df_cumul_failure.columns = ['failure', 'machine', 'day']
df_cumul_failure = df_cumul_failure.reset_index(drop=True)

# detailed maintenance cost
df_cost_maintenance_detail['filter'] = df_cost_maintenance_detail.apply(lambda x: df_maintenance.maintenance[x['machine'],x['day']]==1, axis=1)
df_cost_maintenance_detail = df_cost_maintenance_detail[df_cost_maintenance_detail['filter'] == True]
df_cost_maintenance_detail.drop(['filter'], axis=1, inplace=True)

df_maintenance = df_maintenance.reset_index(drop=True)

# maintenance kpi: cost breakdown
df_cost_maint_sum = df_cost_maintenance_detail.drop(['machine', 'day'], axis=1)
cost_maint_vals = df_cost_maint_sum.sum().reset_index().values
df_maint_kpis = pd.DataFrame(cost_maint_vals, columns=['kpi', 'value'])

outputs = {}
    
outputs['cost_maintenance'] = df_cost_maintenance
outputs['cumul_failure'] = df_cumul_failure
outputs["kpis"] = df_kpis
outputs["maint_kpis"] = df_maint_kpis
outputs["maintenance"] = df_maintenance
outputs["production"] = df_production
outputs["production_shortage"] = df_production_shortage
outputs["resource_load"] = df_resource_load
outputs['maintenance_cost_detail'] = df_cost_maintenance_detail


Appending to model/main.py


In [57]:
import tarfile
def reset(tarinfo):
    tarinfo.uid = tarinfo.gid = 0
    tarinfo.uname = tarinfo.gname = "root"
    return tarinfo
tar = tarfile.open("model.tar.gz", "w:gz")
tar.add("model/main.py", arcname="main.py", filter=reset)
tar.close()

# Upload your model on Watson Machine Learning

Store model in Watson Machine Learning with:

    the tar archive previously created,
    metadata including the model type and runtime

Get the model_uid.


In [58]:
mnist_metadata = {
    client.repository.ModelMetaNames.NAME: "PM Opti",
    client.repository.ModelMetaNames.DESCRIPTION: "Preventive Maintenance Opti Model",
    client.repository.ModelMetaNames.TYPE: "do-docplex_12.9",
    client.repository.ModelMetaNames.RUNTIME_UID: "do_12.9"    
}

model_details = client.repository.store_model(model='/home/dsxuser/work/model.tar.gz', meta_props=mnist_metadata)

model_uid = client.repository.get_model_uid(model_details)

print( model_uid )

15db426b-8dee-41d7-9b1f-1717efff0e0c


# Create a deployment

Create a batch deployment for the model, providing information such as:

    the maximum number of compute nodes
    the T-shirt size of the compute nodes

Get the deployment_uid.

In [59]:
meta_props = {
    client.deployments.ConfigurationMetaNames.NAME: "PM Deployment",
    client.deployments.ConfigurationMetaNames.DESCRIPTION: "PM DO Deployment",
    client.deployments.ConfigurationMetaNames.BATCH: {},
    client.deployments.ConfigurationMetaNames.COMPUTE: {'name': 'S', 'nodes': 1}
}

deployment_details = client.deployments.create(model_uid, meta_props=meta_props)

deployment_uid = client.deployments.get_uid(deployment_details)

print( deployment_uid )



#######################################################################################

Synchronous deployment creation for uid: '15db426b-8dee-41d7-9b1f-1717efff0e0c' started

#######################################################################################


ready.


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='8baf07ca-bb9a-4006-b5ed-0c901b0bb3a1'
------------------------------------------------------------------------------------------------


8baf07ca-bb9a-4006-b5ed-0c901b0bb3a1


In [60]:
# List all existing deployments

client.deployments.list()

------------------------------------  ---------------------------------------------------  -----  ------------------------  -------------
GUID                                  NAME                                                 STATE  CREATED                   ARTIFACT_TYPE
8baf07ca-bb9a-4006-b5ed-0c901b0bb3a1  PM Deployment                                        ready  2019-11-07T23:33:07.181Z  model
7640fe91-9263-4da1-b05e-4a5fd8ddfcf4  PM Deployment                                        ready  2019-11-07T22:57:57.490Z  model
9e4a67d8-1468-49fc-b068-569c01e6b74c  Predictive Maintenance Model Py36 Online Deployment  ready  2019-11-07T20:05:41.170Z  model
7ceaa300-9cb4-4307-a64b-f01bd00fae59  Auto generated DO docplex deployment                 ready  2019-10-25T14:53:04.628Z  model
6b488d5b-e2c6-4ffa-9cda-e64c931bb195  api2                                                 ready  2019-02-15T00:25:31.594Z  model
fe6ec487-0ca0-4058-9de7-39ce21f173c2  im an api                           

# Create and monitor a job with inline data for your deployed model¶

Create a payload containing inline input data.

Create a new job with this payload and the deployment.

Get the job_uid

In [61]:
solve_payload = {
    client.deployments.DecisionOptimizationMetaNames.INPUT_DATA_REFERENCES: [
                  {
                    "id":"day.csv",
                    "type": "s3",
                    "connection": {
                        "endpoint_url": "s3-api.us-geo.objectstorage.service.networklayer.com",
                        "access_key_id": "2ed76c071b6448ff9f603f30b3431b8a",
                        "secret_access_key": "4102030211f936020bf8de5391bfe21e2ca36d23265a6ba3"
                    },
                    "location": {
                        "bucket": "predictivemaintenance-donotdelete-pr-mlejlc5nq1ttru",
                        "path": "day.csv"
                    }
                },
                {
                    "id":"machine.csv",
                    "type": "s3",
                    "connection": {
                        "endpoint_url": "s3-api.us-geo.objectstorage.service.networklayer.com",
                        "access_key_id": "2ed76c071b6448ff9f603f30b3431b8a",
                        "secret_access_key": "4102030211f936020bf8de5391bfe21e2ca36d23265a6ba3"
                    },
                    "location": {
                        "bucket": "predictivemaintenance-donotdelete-pr-mlejlc5nq1ttru",
                        "path": "machine.csv"
                    }
                },
                {
                    "id":"predicted_failure.csv",
                    "type": "s3",
                    "connection": {
                        "endpoint_url": "s3-api.us-geo.objectstorage.service.networklayer.com",
                        "access_key_id": "2ed76c071b6448ff9f603f30b3431b8a",
                        "secret_access_key": "4102030211f936020bf8de5391bfe21e2ca36d23265a6ba3"
                    },
                    "location": {
                        "bucket": "predictivemaintenance-donotdelete-pr-mlejlc5nq1ttru",
                        "path": "predicted_failure.csv"
                    }
                },
                {
                    "id":"planned_production.csv",
                    "type": "s3",
                    "connection": {
                        "endpoint_url": "s3-api.us-geo.objectstorage.service.networklayer.com",
                        "access_key_id": "2ed76c071b6448ff9f603f30b3431b8a",
                        "secret_access_key": "4102030211f936020bf8de5391bfe21e2ca36d23265a6ba3"
                    },
                    "location": {
                        "bucket": "predictivemaintenance-donotdelete-pr-mlejlc5nq1ttru",
                        "path": "planned_production.csv"
                    }
                },
                {
                    "id":"parameters.csv",
                    "type": "s3",
                    "connection": {
                        "endpoint_url": "s3-api.us-geo.objectstorage.service.networklayer.com",
                        "access_key_id": "2ed76c071b6448ff9f603f30b3431b8a",
                        "secret_access_key": "4102030211f936020bf8de5391bfe21e2ca36d23265a6ba3"
                    },
                    "location": {
                        "bucket": "predictivemaintenance-donotdelete-pr-mlejlc5nq1ttru",
                        "path": "parameters.csv"
                    }
                },
                {
                    "id":"fixed_maintenance.csv",
                    "type": "s3",
                    "connection": {
                        "endpoint_url": "s3-api.us-geo.objectstorage.service.networklayer.com",
                        "access_key_id": "2ed76c071b6448ff9f603f30b3431b8a",
                        "secret_access_key": "4102030211f936020bf8de5391bfe21e2ca36d23265a6ba3"
                    },
                    "location": {
                        "bucket": "predictivemaintenance-donotdelete-pr-mlejlc5nq1ttru",
                        "path": "fixed_maintenance.csv"
                    }
                }
            ],
        client.deployments.DecisionOptimizationMetaNames.OUTPUT_DATA: [
            {
                "id":".*\\.csv"
            }
       ]
    }

In [62]:
job_details = client.deployments.create_job(deployment_uid, solve_payload)
job_uid = client.deployments.get_job_uid(job_details)

print( job_uid )

84e4b2a7-72c5-4bd4-90ef-ec146974f42a


In [63]:
from time import sleep

while job_details['entity']['decision_optimization']['status']['state'] not in ['completed', 'failed', 'canceled']:
    print(job_details['entity']['decision_optimization']['status']['state'] + '...')
    sleep(5)
    job_details=client.deployments.get_job_details(job_uid)

print( job_details['entity']['decision_optimization']['status']['state'])

queued...
queued...
queued...
queued...
queued...
running...
running...
completed


In [64]:
print( job_details['entity']['decision_optimization']['status'])

{'state': 'completed', 'running_at': '2019-11-07T23:33:44.168Z', 'completed_at': '2019-11-07T23:33:47.149Z'}


In [65]:
client.deployments.delete(deployment_uid)

'SUCCESS'